https://people.eecs.berkeley.edu/~vazirani/algorithms/chap2.pdf

# 2.1: Multiplication
## Imaginary nums
$(a+bi)(c+di) = ac - bd + (ad + bc)i$   
$ad + bc = (a+b)(c+d) - ac - bd$  
4 -> 3 multiplies.

$n$-bit add is $O(n)$, grade-school multiply is $O(n^2)$

## General case
$x = x_L x_R = 2^{\frac{n}{2}} x_L + x_R$  
$y = y_L y_R = 2^{\frac{n}{2}} y_L + y_R$  

Without Gauss:  
$xy = (2^{\frac{n}{2}} x_L + x_R)(2^{\frac{n}{2}} y_L + y_R) = 2^n x_L y_L + x_R y_R + 2^{\frac{n}{2}} (x_L y_R + x_R y_L)$  
$T(n) = 4 T(\frac{n}{2}) + O(n)$

With Gauss:  
$xy = 2^n x_L y_L + x_R y_R + 2^{\frac{n}{2}} ((x_L + x_R)(y_L + y_R) - x_L y_L - x_R y_R)$  
$T(n) = 3 T(\frac{n}{2}) + O(n)$

## Runtimes
Subproblems are halved in size every recursive call. At $\log_2(n)$ level, subproblems are size 1, recursion ends. Height of tree is $\log_2(n)$.

Without Gauss
* Level $k$, $4^k$ subproblems, each of size $O(\frac{n}{2^k})$, work is $4^k * O(\frac{n}{2^k}) = 2^k O(n)$.
* $\text{Work} = \sum_{k=0}^{\log_2(n)} 2^k O(n)$. Since $2>1$, the work is dominated by the work at the very bottom.
* At the bottom: $\text{Work} = 2^{\log_2(n)} O(n) = O(n^2)$

With Gauss
* Level $k$, $3^k$ subproblems, each of size $O(\frac{n}{2^k})$, work is $3^k * O(\frac{n}{2^k}) = (\frac{3}{2})^k O(n)$.
* $\text{Work} = \sum_{k=0}^{\log_2(n)} (\frac{3}{2})^k O(n)$. Since $\frac{3}{2}>1$, the work is dominated by the work at the very bottom.
* At the bottom: $\text{Work} = (\frac{3}{2})^{\log_2(n)} O(n) = 2^{\log_2(\frac{3}{2})^{\log_2(n)}} O(n) = n^{\log_2(\frac{3}{2})} O(n) \approx O(n^{1.58})$

Implementation: don't recurse down to 1 bit since 16 or 32 bit multiplication is a single operation on CPUs.

# 2.2: Master Theorem
Divide and conquer: solve problem of size $n$ by recursively solving $a$ subproblems of size $\frac{n}{b}$ and combining in $O(n^d)$ time.  
$T(n) = a T(\frac{n}{b}) + O(n)^d$

\# Levels: $\frac{n}{b^k} = 1 \to n = b^k \to k = \log_b(n)$  
\# Subproblems for level $k$: $a^k$  
\# Size of subproblem for level $k$: $O(\frac{n}{b^k})^d$

$T(n) = \sum_{k=0}^{\log_b(n)} a^k \times O(\frac{n}{b^k})^d = \sum_{k=0}^{\log_b(n)} \frac{a^k}{b^{dk}} \times O(n^d) = \sum_{k=0}^{\log_b(n)} (\frac{a}{b^d})^k \times O(n^d)$
* $\frac{a}{b^d} > 1$: bottom heavy, $(\frac{a}{b^d})^{\log_b(n)} \times O(n^d) = b^{\log_b(\frac{a}{b^d}) \times \log_b(n)} \times O(n^d) = n^{\log_b(\frac{a}{b^d})} \times O(n^d) = n^{\log_b(\frac{a}{b^d})} \times O(n^d) = n^{\log_b(a) - d} \times O(n^d) = O(n^{\log_b(a)})$
* $\frac{a}{b^d} = 1$ runtime same at every level: $\log_b(n) \times O(n^d) = O(n^d \log_b(n))$
* $\frac{a}{b^d} = 1$: top heavy, $O(n^d)$

# 2.3: Mergesort
Given an unsorted array, divide the array in half, recursively sort each half, the merge both halves together.

Merging 2 sorted arrays together takes $O(n)$ linear time.

$T(n) = 2T(\frac{n}{2}) + O(n)$  
$T(n) = \sum_{k=o}^{\log_2(n)} 2^k O(\frac{n}{2^k}) = \sum_{k=o}^{\log_2(n)} O(n) = O(n \log_2(n))$

In [1]:
def mergesort(L: list[int]) -> None:
    if len(L) <= 1:
        return L
    
    mid = len(L) // 2
    left, right = L[:mid], L[mid:]
    
    mergesort(left)
    mergesort(right)

    # Merge.
    i, j = 0, 0
    while (i < len(left)) and (j < len(right)):
        if left[i] < right[j]:
            L[i+j] = left[i]
            i += 1
        else:
            L[i+j] = right[j]
            j += 1

    L[i+j:] = left[i:] + right[j:]
    
L = [4, 2, 1, 6, 32, 3, 9, 7]
mergesort(L)
print(L)

[1, 2, 3, 4, 6, 7, 9, 32]


Comparison-based sorting can be represented as a binary tree with leaves having permutations of the array (one of them is sorted).

Tree sorts array of $n$ elements. Leaves are every permutation of ${1, 2, ..., n}$. $n!$ permutations, therefore $n!$ leaves.  
Binary tree with $n$ leaves has depth of $\log_2(n)$. Tree must then have a depth (and runtime) of at most O($\log_2(n!)) = O(n \log_2(n))$

Only applies to comparison-based sorting (not Radix and Counting Sort, which only work for integers).

# 2.4: Medians
Naive: Sort $O(n \log(n))$, then pick middle element.

Selection: get the $k$-th smallest element of $S$. Pick a number $v$ and split all elements in $S$ into $S_L$ (elements < $v$), $S_v$ (elements = $v$), and $S_R$ (elements > $v$).

$S = \{ 2, 36, 5, 21, 8, 13, 11, 20, 5, 4, 1 \}$  
$v = 5$  
$S_L = \{ 2, 4, 1 \}$, $S_v = \{ 5, 5 \}$, $S_R = \{ 36, 21, 8, 13, 11, 20 \}$

If we want the $8$-th smallest element of $S$, we immediately know that we need the $3$-rd smallest element of $S_R$. 

$
\text{selection}(S, k) = \begin{cases} 
    \text{selection}(S_L, k) & \text{if} & k \le |S_L| \\
    v & \text{if} & |S_L| < k \le |S_R| \\
    \text{selection}(S_R, k - |S_L| - |S_v|) & \text{if} & k > |S_L| + |S_v| \\
\end{cases}
$

Computing $S_L, S_v, S_R$ is linear time. Selection reduces elements from $|S|$ to $\max(|S_L|, |S_R|)$.  
Ideally we would pick $v$ so that $|S_L| = |S_R| = \frac{1}{2} |S|$, cutting the search space in half each time, $T(n) = 2T(\frac{n}{2}) + O(n)$.  
However, $v$ would have to be the median for that to be true, so instead we pick randomly.

Worst case for randomly picking $v$ is picking the min/max, cutting down only by 1 each time. $n + (n-1) + ... + \frac{n}{2} = \Theta(n^2)$.  
Best case of picking exactly the median is equally unlikely. $O(n)$.

$v$ is good if lies in $25-75$-th percentile (remaining search space will be at most $\frac{3}{4}$ the size).  
$v$ has a $50%$ change of being good. $v$ will be good an average every $2$ picks.  
After 2 splits on average, the search space will shrink to $\frac{3}{4}$ its size: $T(n) \le T(\frac{3}{4} n) + O(n)$  
$T(n) = O(n)$.

Quicksort: random split (partitioning the array), and sorting each partition. Also $O(n \log(n))$, but empirically faster than merge sort.

# 2.5: Matrix Multiplication
$XY = Z$, where $X, Y, Z$ are all $(n, n)$ matricies. Naively takes $O(n^3)$ operations, fill in all $n^2$ elements of $Z$, where each element is a $O(n)$ dot product.

Naive divide and conquer:  
$X = \begin{bmatrix} A & B \\ C & D \\ \end{bmatrix}, Y = \begin{bmatrix} E & F \\ G & H \\ \end{bmatrix}$  
$XY = \begin{bmatrix} AE + BG & AF  + BH \\ CE + DG & CF + DH \\ \end{bmatrix}$

$T(n) = 8T(\frac{n}{2}) + O(n^2)$  
$T(n) = \sum_{k=0}^{\log_2(n)} 8^k * O(\frac{n}{2^k})^2 = \sum_{k=0}^{\log_2(n)} 2^k * O(n^2) = O(n^3)$

Strassen:  
$XY$ only requires computing 7 $(\frac{n}{2}, \frac{n}{2})$ subproblems, not 8. (Quite a mess of algebra).

$T(n) = 7T(\frac{n}{2}) + O(n^2)$  
$T(n) = \sum_{k=0}^{\log_2(n)} 7^k * O(\frac{n}{2^k})^2 = \sum_{k=0}^{\log_2(n)} (\frac{7}{4})^k * O(n^2) = O(n^{\log_2(\frac{7}{4})}) * O(n^2) \approx O(n^{2.81})$

# 2.6: FFT
Polynomial multiplication: $(1+2x)(2+x) = 2+5x+2x^2$  
Product of 2 $d$-degree polynomials is a polynomial of degree $2d$  

$A(x) = a_0 + a_1 x + ... + a_d x^d, B(x) = b_0 + b_1 x + ... + b_d x^d$  
$C(x) = A(x)B(x) = c_0 + c_1 x + ... + c_{2d} x^{2d}, c_k = a_0 b_k + a_1 b_{k-1} + ... + a_k b_0$

Naive time complexity of $\Theta(d^2)$

## Importance
Int multiplication is polynomial multiplication (polynomials and binary int similarity).

System(Signal) = Response  
Linear: System(Signal1 + Signal2) = System(Signal1) + System(Signal2)  
Time Invariant: System(Signal shifted by time t) = Response shifted by time t  

LTI systems completely determined by system response of unit signal (only shift and scale).  
System response $c(t) = \sum_{i=0}^t a(i) \cdot b(t-i)$, summing input $a(i)$ times unit system response $b(t-i)$ after elapsed time since impulse. Same as polynomial multiplication.

## Alt polynomial repr
$d$-degree polynomial uniquely characterized by values at $d+1$ distinct points (2 points for line, 3 points for quadratic...).

Coeffs --evaluate--> values at points --interpolate--> coeffs.

Evalutating polynomial of degree $d \le n$ at 1 step takes $O(n)$ steps, so evaluating it at $n$ points takes $\Theta(n^2)$ time. 

Input: Coeffs of $A(x), B(x)$ of degree $d$  
1. Select: pick points $x_0, x_1, ..., x_{n-1}$, where $n \ge 2d + 1$
2. Evaluate: compute $A(x_0), A(x_1), ..., A(x_{n-1})$ and $B(x_0), B(x_1), ..., B(x_{n-1})$ 
3. Multipy: compute $C(x_k) = A(x_k)B(x_k), k \in \{0, ..., n-1\}$
4. Interpolate: recover coeffs $C(x) = c_0 + c_1 x + ... + c_{2d} x^{2d}$

## Evaluation w/ divide and conquer
Pick positive-negative pairs, $\pm x_0, \pm x_2, ..., \pm x_{\frac{n}{2} - 1}$ to evaluate polynomial $A(x)$ of degree $d \le n-1$.  
Computation for even power terms overlap.  

$3+4x + 6x^2 + 2x^3 + x^4 + 10x^5 = (3 + 6x^2 +x^4) + x(4 + 2x^2 + 10x^4)$  
$A(x) = A_e(x^2) + x A_o(x^2)$  

$A(x_i) = A_e(x^2) + x A_o(x^2)$  
$A(-x_i) = A_e(x^2) - x A_o(x^2)$

Evaluating $A(x)$ at $n$ paired points $\pm x_0, ..., \pm x_{\frac{n}{2} - 1}$ can be computed by evalutating $A_o(x)$ and $A_e(x)$ at $\frac{n}{2}$ points $x_0^2, ..., x_{\frac{n}{2} - 1}^2$.

Recursing: $T(n) = 2 T(n/2) + O(n) = O(n \log(n))$  
We need an additional trick to recurse since we need $\pm$ pairs.

Bottom of recursion is $1$, next level is $1, -1$, next level is $1, -1, i, -i$, ...  
Final level is the complex $n$-th roots of unity, $n$ complex solutions to $z^n = 1$.

$z = a + bi = r(cos(\theta) + i sin(\theta)) = r e^{i \theta}$  
$z_1 \cdot z_2 = r_1 e^{i \theta_1} \cdot r_2 e^{i \theta_2} = r_1 r_2 e^{i (\theta_1 + \theta_2)} = (r_1 r_2, \theta_1 + \theta_2)$  
$-z = (r, \theta + \pi)$  
$z$ on unit circle: $r=1, z^n = (1, n\theta)$

$n$-th complex roots of unity: $n$-solutions to $z^n = 1$.  
$z = (1, \theta), \theta = \frac{2 \pi}{n}$  
$\pm$ paired, $-(1, \theta) = (1, \theta + \pi)$  
Squares of pairs are $\frac{n}{2}$ roots of unity

![Complex Roots of Unity](images/complex_roots_of_unity.png)

FFT(Coeffs of $A(x)$ of degree $d \le n-1$, $\omega$ an nth root of unity) -> Value repr $A(\omega^0), ..., A(\omega^{n-1})$
* if $\omega = 1$: base case, return $A(1)$
* Divide: $A(x) = A_e(x^2) + xA_o(x^2)$
* Conquer: Call $\text{FFT}(A_e, \omega^2), \text{FFT}(A_o, \omega^2)$, at even powers of $\omega$
* Combine: compute $A(\omega^i) = A_e(\omega^{2i}) + \omega A_o(\omega^{2i})$, note that only the sign of $\omega A_o(\omega^{2i})$ flips b/t even and odd powers of $\omega$
* Return $A(\omega^0), ..., A(\omega^{n-1})$

## Interpolation
$\text{values} = \text{FFT}(\text{coeffs}, \omega)$  
$\text{coeffs} = \text{FFT}(\text{values}, \omega^{-1})$  

$ A = \begin{bmatrix} A(x_0) \\
A(x_1) \\
\vdots \\
A(x_{n-1})
\end{bmatrix}, M = \begin{bmatrix}
1, x_0, x_0^2, \dots, x_0^{n-1}  \\
1, x_1, x_1^2, \dots, x_1^{n-1} \\
\vdots \\
1, x_{n-1}, x_{n-1}^2, \dots, x_{n-1}^{n-1}
\end{bmatrix}, a = \begin{bmatrix} 
a_0 \\
a_1 \\
\vdots \\
a_{n-1}
\end{bmatrix} $  
$A = Ma$  
$M$ is a Vandermonde matrix, invertible if $x_0, ..., x_{n-1}$ are distinct.  
Evaluation is $A = Ma$, interpolation is $a = M^{-1} A$

$M(\omega) = \begin{bmatrix}
1, 1, 1, \dots, 1  \\
1, \omega, \omega^2, \dots, \omega^{n-1} \\
\vdots \\
1, \omega^{n-1}, \omega^{2(n-1)}, \dots, \omega^{(n-1)(n-1)} \\
\end{bmatrix}$  
$M(\omega)[i,j] = \omega^{ij}$

Inner product of columns $j$ and $k$: $1 + w^{j-k} + w^{2(j-k)} + ... + w^{(n-1)(j-k)} = \frac{1 - (w^{(j-k)})^n}{1 - w^{(j-k)}} = \begin{cases} 0 & \text{if} & j \neq k \\ n & \text{if} & j = k\end{cases}$.  
Note that the inner product between complex vectors is $u \cdot v^*$, where $v^*$ is the complex conjugate $v^* = e^{-i \theta}$.  
Therefore columns of $M$ are orthogonal to each other, $M$ is the Fourier basis, a rigid rotation.  
$MM^* = nI \to M^{-1} = M(w^{-1})$, rows and cols are orthogonal because $M$ is its own transpose.  

Rotate vectors into Fourier basis (evalutate), perform task (multiply), then rotate back (interpolate).

## FFT algorithm
* Input: $a = (a_0, ..., a_{n-1})$ and $w$, whose powers $1, w, w^2, ..., w^{n-1}$ are the complex $n$-th roots of unity.
* Multiply: $A = Ma$, where $A$ is the value representation of the product polynomial, and $M[j,k] = w^{jk}$
    * Divide: 